In [3]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
import abc

In [ ]:
class Kernel(abc.ABC):
    def __init__(self, bw=1.0):
        self.bw = float(bw)
        
    @abc.abstractmethod
    def fit(self, X):
        raise NotImplemented
    @abc.abstractmethod
    def score_samples(self, P):
        raise NotImplemented

In [6]:
class GaussianKernel(Kernel):
    
    def fit(self, X):
        self.X = np.asarray(X, float)
        self.n_samples, self.n_features = self.X.shape
        self.X_norm_sq = np.sum(self.X**2, axis=1)
        return self

    def score_samples(self, points):
        P = np.atleast_2d(points).astype(float)
        P_norm_sq = np.sum(P**2, axis=1)
        D2 = P_norm_sq[:, None] + self.X_norm_sq[None, :] - 2 * P.dot(self.X.T)
        K = np.exp(-0.5 * D2 / self.bandwidth**2)
        norm = (2*np.pi)**(self.n_features/2) * self.bandwidth**self.n_features * self.n_samples
        return K.sum(axis=1) / norm

In [7]:
class KDE(BaseEstimator):
    '''
    bandwidth: float
        The bandwidth of the kernel.
    '''
    
    def __init__(self, bandwidth=1.0):
        self.bandwidth = bandwidth
        self.kernel = 'gaussian'
        self.kernel_obj = None
        
    def set_kernel(self, kernel):
        if kernel not in ['gaussian', 'tophat', 'epanechnikov', 'exponential', 'linear', 'cosine']:
            raise ValueError("invalid kernel type")
        self.kernel = kernel
        if kernel == 'gaussian':
            self.kernel_obj = GaussianKernel(self.bandwidth)
        # // TODO: implement other kernels
        
    
    def fit(self, X, y=None):
        '''
        Fit KDE model.
        
        X: DF
        y: array-like of target variable
        '''
        
        self.kernel_obj.fit(X)
        
    def score_samples(self, points):
        ''' 
        Compute density for points.
        returns array[m]
        '''
        return self.kernel_obj.score_samples(points)
        

        
        